In [455]:
import pandas as pd
import re
from pathlib import Path  

Read in large NBA Data

In [456]:
data = pd.read_excel('/content/nba_players_2023.xls', header=1, index_col=2)
data

,Unnamed: 0,#,Pos,HT,WT,Age,Current Team,YOS,Pre-Draft Team,Draft Status,Nationality
Player,,,,,,,,,,,
Udonis Haslem,188,40,FC,6-8,235,42,Miami Heat,19,Florida,"2002 NBA Draft, Undrafted",United States
Andre Iguodala,212,9,SF,6-6,215,39,Golden State Warriors,18,Arizona,2004 Rnd 1 Pick 9,United States
LeBron James,222,6,F,6-8,250,38,Los Angeles Lakers,19,St. Vincent St. Mary High School (Ohio),2003 Rnd 1 Pick 1,United States
Taj Gibson,149,67,F,6-9,232,37,Washington Wizards,13,USC,2009 Rnd 1 Pick 26,United States
P.J. Tucker,454,17,SF,6-6,245,37,Philadelphia Sixers,11,Texas,2006 Rnd 2 Pick 5,United States
...,...,...,...,...,...,...,...,...,...,...,...
Trevor Keels,245,0,PG,6-4,221,19,New York Knicks,0,Duke,2022 Rnd 2 Pick 12,United States
Nikola Jovic,241,5,F,6-11,223,19,Miami Heat,0,KK Mega Bemax (Serbia),2022 Rnd 1 Pick 27,Serbia
Ousmane Dieng,113,13,SG,6-10,216,19,Oklahoma City Thunder,0,New Zealand (New Zealand),2022 Rnd 1 Pick 11,France


Select only the player, position, height, age, current team, draft status and nationality columns and re-bucket data for analysis

In [457]:
#keeping player positions, height, age, draft status, nationality, team
df = data.loc[:,['Pos','HT','Age','Current Team','Draft Status', 'Nationality']]

#replace positions
df['Pos'] = df['Pos'].replace(['F','SF','PF'], "Forward")
df['Pos'] = df['Pos'].replace(['C','FC'], "Center")
df['Pos'] = df['Pos'].replace(['SG','PG','GF','G'], "Guard")

#replace heights 
df['HT'] = df['HT'].replace(['5-8','5-9','5-10','5-11'], "<6")
df['HT'] = df['HT'].replace(['6-0','6-1','6-2','6-3','6-4','6-5'], "6-6_5")
df['HT'] = df['HT'].replace(['6-6','6-7','6-8','6-9','6-10','6-11'], "6_6-6_11")
df['HT'] = df['HT'].replace(['7-0','7-1','7-2','7-3','7-4','7-5','7-6'], "7+")

#replace ages
df['Age'] = df['Age'].replace([19,20,21,22,23,24], '<25')
df['Age'] = df['Age'].replace([25,26,27,28,29], '25-29')
df['Age'] = df['Age'].replace([30,31,32,33,34], '30-34')
df['Age'] = df['Age'].replace([35,36,37,38,39,40,41,42,43,44], '35+')


#replace teams with respective conference
df['Conference']=df['Current Team']
df['Conference'] = df['Conference'].replace(['Miami Heat','Milwaukee Bucks','Cleveland Cavaliers','Washington Wizards', 'Charlotte Hornets','Chicago Bulls','New York Knicks','Atlanta Hawks','Brooklyn Nets','Boston Celtics','Philadelphia Sixers','Detroit Pistons', 'Orlando Magic', 'Indiana Pacers'], 'Eastern')
df['Conference'] = df['Conference'].replace(['Golden State Warriors','Los Angeles Lakers', 'Phoenix Suns', 'Utah Jazz','Denver Nuggets','New Orleans Pelicans','Dallas Mavericks', 'Minnesota Timberwolves', 'Houston Rockets', 'Los Angeles Clippers', 'Toronto Raptors', 'San Antonio Spurs', 'Sacramento Kings', 'Portland Trail Blazers','Memphis Grizzlies','Oklahoma City Thunder'], 'Western')

#replace draft status
df.replace(r'^.*(Rnd 1|Round 1).*$', value='Round_1', inplace=True, regex=True)
df.replace(r'^.*(Rnd 2|Round 2).*$', value='Round_2', inplace=True, regex=True)
df.replace(r'^.*(Undrafted).*$', value='Undrafted', inplace=True, regex=True)

#replace nationality 
df.replace(r'^.*(United States).*$', value='Domestic', inplace=True, regex=True)
df.loc[df["Nationality"] != "Domestic", "Nationality"] = 'International'

#add All_star column
df['All_Star']='All_Star_No'
df.loc[['Kyrie Irving','Donovan Mitchell', 'Giannis Antetokounmpo','Kevin Durant','Jayson Tatum','Jaylen Brown','DeMar DeRozan','Tyrese Haliburton','Jrue Holiday','Julius Randle','Bam Adebayo','Joel Embiid','Pascal Siakam', 'Stephen Curry', 'Luka Doncic', 'Nikola Jokic','LeBron James','Zion Williamson','Shai Gilgeous-Alexander','Damian Lillard','Ja Morant', 'Paul George','Jaren Jackson, Jr.','Lauri Markkanen','Domantas Sabonis','Anthony Edwards',"De'Aaron Fox"],'All_Star']='All_Star_Yes'

#rename columns 
df = df.rename(columns={"Pos":"Position","HT":"Height","Draft Status":"Draft","Current Team":"Team"})

In [458]:
df

,Position,Height,Age,Team,Draft,Nationality,Conference,All_Star
Player,,,,,,,,
Udonis Haslem,Center,6_6-6_11,35+,Miami Heat,Undrafted,Domestic,Eastern,All_Star_No
Andre Iguodala,Forward,6_6-6_11,35+,Golden State Warriors,Round_1,Domestic,Western,All_Star_No
LeBron James,Forward,6_6-6_11,35+,Los Angeles Lakers,Round_1,Domestic,Western,All_Star_Yes
Taj Gibson,Forward,6_6-6_11,35+,Washington Wizards,Round_1,Domestic,Eastern,All_Star_No
P.J. Tucker,Forward,6_6-6_11,35+,Philadelphia Sixers,Round_2,Domestic,Eastern,All_Star_No
...,...,...,...,...,...,...,...,...
Trevor Keels,Guard,6-6_5,<25,New York Knicks,Round_2,Domestic,Eastern,All_Star_No
Nikola Jovic,Forward,6_6-6_11,<25,Miami Heat,Round_1,International,Eastern,All_Star_No
Ousmane Dieng,Guard,6_6-6_11,<25,Oklahoma City Thunder,Round_1,International,Western,All_Star_No


Check that all columns were successfully altered

In [459]:
print('Positions:',df.Position.unique())
print('Heights:',df.Height.unique())
print('Ages:',df.Age.unique())
print('Conferences:',df.Conference.unique())
print('Draft Status:',df['Draft'].unique())
print('Nationality:',df.Nationality.unique())
print('All_Star:',df.All_Star.unique())
print('Team:', df.Team.unique())

Positions: ['Center' 'Forward' 'Guard']
Heights: ['6_6-6_11' '6-6_5' '7+' '<6']
Ages: ['35+' '30-34' '25-29' '<25']
Conferences: ['Eastern' 'Western']
Draft Status: ['Undrafted' 'Round_1' 'Round_2']
Nationality: ['Domestic' 'International']
All_Star: ['All_Star_No' 'All_Star_Yes']
Team: ['Miami Heat' 'Golden State Warriors' 'Los Angeles Lakers'
 'Washington Wizards' 'Philadelphia Sixers' 'Phoenix Suns'
 'Boston Celtics' 'Utah Jazz' 'Denver Nuggets' 'Indiana Pacers'
 'Milwaukee Bucks' 'New Orleans Pelicans' 'Cleveland Cavaliers'
 'Dallas Mavericks' 'Minnesota Timberwolves' 'Houston Rockets'
 'Los Angeles Clippers' 'Detroit Pistons' 'Toronto Raptors'
 'New York Knicks' 'Chicago Bulls' 'Brooklyn Nets' 'Charlotte Hornets'
 'San Antonio Spurs' 'Sacramento Kings' 'Portland Trail Blazers'
 'Orlando Magic' 'Atlanta Hawks' 'Memphis Grizzlies'
 'Oklahoma City Thunder']


In [460]:
df.Position.value_counts()

Guard      227
Forward    214
Center      68
Name: Position, dtype: int64

In [461]:
df.Height.value_counts()

6_6-6_11    282
6-6_5       182
7+           42
<6            3
Name: Height, dtype: int64

In [462]:
df.Age.value_counts()

<25      227
25-29    169
30-34     90
35+       23
Name: Age, dtype: int64

In [463]:
df['Conference'].value_counts()

Western    271
Eastern    238
Name: Conference, dtype: int64

In [464]:
df['Draft'].value_counts()

Round_1      274
Undrafted    122
Round_2      113
Name: Draft, dtype: int64

In [465]:
df['Nationality'].value_counts()

Domestic         402
International    107
Name: Nationality, dtype: int64

Apply one-hot encoding function

In [466]:
df

,Position,Height,Age,Team,Draft,Nationality,Conference,All_Star
Player,,,,,,,,
Udonis Haslem,Center,6_6-6_11,35+,Miami Heat,Undrafted,Domestic,Eastern,All_Star_No
Andre Iguodala,Forward,6_6-6_11,35+,Golden State Warriors,Round_1,Domestic,Western,All_Star_No
LeBron James,Forward,6_6-6_11,35+,Los Angeles Lakers,Round_1,Domestic,Western,All_Star_Yes
Taj Gibson,Forward,6_6-6_11,35+,Washington Wizards,Round_1,Domestic,Eastern,All_Star_No
P.J. Tucker,Forward,6_6-6_11,35+,Philadelphia Sixers,Round_2,Domestic,Eastern,All_Star_No
...,...,...,...,...,...,...,...,...
Trevor Keels,Guard,6-6_5,<25,New York Knicks,Round_2,Domestic,Eastern,All_Star_No
Nikola Jovic,Forward,6_6-6_11,<25,Miami Heat,Round_1,International,Eastern,All_Star_No
Ousmane Dieng,Guard,6_6-6_11,<25,Oklahoma City Thunder,Round_1,International,Western,All_Star_No


In [467]:
#put together a large dataset of all active NBA players
df_large = df
large_player_filepath = Path('nba_players_large_v2.xlsx')  
large_player_filepath.parent.mkdir(parents=True, exist_ok=True)  
df_large.to_excel(large_player_filepath)  

In [468]:
#one-hot a large dataset of all active NBA players
one_hot_large = pd.get_dummies(df_large)
one_hot_large = one_hot_large.rename(columns={'Position_Center':'Center','Position_Forward':'Forward',"Position_Guard":"Guard","Height_6-6_5":"6-6_5","Height_6_6-6_11":"6_6-6_11","Height_7+":"7+","Height_<6":"<6",'Age_25-29':'25-29','Age_30-34':'30-34','Age_35+':'35+',"Draft_Round_1":"Round_1","Draft_Round_2":"Draft_2","Draft_Undrafted":"Undrafted","Nationality_Domestic":"Domestic","Nationality_International":"International","Conference_Eastern":"Eastern","Conference_Western":"Western","All_Star_All_Star_No":"All_Star_No","All_Star_All_Star_Yes":"All_Star_Yes"})
one_hot_large = one_hot_large.rename(columns=lambda x: re.sub('^Team_','',x))
large_one_hot_filepath = Path('NBA_features2Hidden_Large_v2.xlsx')  
large_one_hot_filepath.parent.mkdir(parents=True, exist_ok=True)  
one_hot_large.to_excel(large_one_hot_filepath) 

In [469]:
#put together a medium dataset of only the eastern conference
df_medium = df[df['Conference']=='Eastern']
df_medium = df.drop(columns=['Conference'])
medium_player_filepath = Path('nba_players_medium_v2.xlsx')  
medium_player_filepath.parent.mkdir(parents=True, exist_ok=True)  
df_medium.to_excel(medium_player_filepath) 

In [470]:
#one-hot a medium dataset of only the eastern conference
one_hot_medium = pd.get_dummies(df_medium)
one_hot_medium = one_hot_medium.rename(columns={'Position_Center':'Center','Position_Forward':'Forward',"Position_Guard":"Guard","Height_6-6_5":"6-6_5","Height_6_6-6_11":"6_6-6_11","Height_7+":"7+","Height_<6":"<6",'Age_25-29':'25-29','Age_30-34':'30-34','Age_35+':'35+',"Draft_Round_1":"Round_1","Draft_Round_2":"Draft_2","Draft_Undrafted":"Undrafted","Nationality_Domestic":"Domestic","Nationality_International":"International","All_Star_All_Star_No":"All_Star_No","All_Star_All_Star_Yes":"All_Star_Yes"})
one_hot_medium = one_hot_medium.rename(columns=lambda x: re.sub('^Team_','',x))
medium_one_hot_filepath = Path('NBA_features2Hidden_Medium_v2.xlsx')  
medium_one_hot_filepath.parent.mkdir(parents=True, exist_ok=True)  
one_hot_medium.to_excel(medium_one_hot_filepath) 


In [471]:
#put together a small dataset of only all-stars
#all-stars: 'Kyrie Irving','Donovan Mitchell', 'Giannis Antetokounmpo','Kevin Durant','Jayson Tatum','Jaylen Brown','DeMar DeRozan','Tyrese Haliburton','Jrue Holiday','Julius Randle','Bam Adebayo','Joel Embiid','Pascal Siakam', 'Stephen Curry', 'Luka Doncic', 'Nikola Jokic','LeBron James','Zion Williamson','Shai Gilgeous-Alexander','Damian Lillard','Ja Morant', 'Paul George','Jaren Jackson Jr.','Lauri Markkanen','Domantas Sabonis','Anthony Edwards',"De'Aaron Fox"], 'Nikola Jokic','LeBron James','Zion Williamson','Shai Gilgeous-Alexander','Damian Lillard','Ja Morant', 'Paul George','Jaren Jackson Jr.','Lauri Markkanen','Domantas Sabonis','Anthony Edwards',"De'Aaron Fox" 
df_small = df.loc[['Kyrie Irving','Donovan Mitchell', 'Giannis Antetokounmpo','Kevin Durant','Jayson Tatum','Jaylen Brown','DeMar DeRozan','Tyrese Haliburton','Jrue Holiday','Julius Randle','Bam Adebayo','Joel Embiid','Pascal Siakam', 'Stephen Curry', 'Luka Doncic', 'Nikola Jokic','LeBron James','Zion Williamson','Shai Gilgeous-Alexander','Damian Lillard','Ja Morant', 'Paul George','Jaren Jackson, Jr.','Lauri Markkanen','Domantas Sabonis','Anthony Edwards',"De'Aaron Fox"]]
df_small = df.drop(columns=['All_Star'])
small_player_filepath = Path('nba_players_small_v2.xlsx')  
small_player_filepath.parent.mkdir(parents=True, exist_ok=True)  
df_small.to_excel(small_player_filepath)  

In [472]:
#one-hot the small dataset of only all-stars
one_hot_small = pd.get_dummies(df_small)
one_hot_small = one_hot_small.rename(columns={'Position_Center':'Center','Position_Forward':'Forward',"Position_Guard":"Guard","Height_6-6_5":"6-6_5","Height_6_6-6_11":"6_6-6_11","Height_7+":"7+","Height_<6":"<6",'Age_25-29':'25-29','Age_30-34':'30-34','Age_35+':'35+',"Draft_Round_1":"Round_1","Draft_Round_2":"Draft_2","Draft_Undrafted":"Undrafted","Nationality_Domestic":"Domestic","Nationality_International":"International","Conference_Eastern":"Eastern","Conference_Western":"Western","All_Star_All_Star_No":"All_Star_No","All_Star_All_Star_Yes":"All_Star_Yes"})
one_hot_small = one_hot_small.rename(columns=lambda x: re.sub('^Team_','',x))
small_one_hot_filepath = Path('NBA_features2Hidden_Small_v2.xlsx')  
small_one_hot_filepath.parent.mkdir(parents=True, exist_ok=True)  
one_hot_small.to_excel(small_one_hot_filepath) 